In [ ]:
import numpy as np
import csv

instances = {}
MPC_instances_path = 'MCP_Instances/'
with os.scandir(MPC_instances_path) as inst_list:
    for inst in inst_list:
        inst_path = MPC_instances_path + inst.name
        with open(inst_path) as f:
            reader = csv.reader(f,delimiter=' ')
            data = list(reader)
            instances[inst.name] = {}
            M = int(data[0][0])          
            N = int(data[1][0])            
            max_loads = [int(n) for n in data[2]]   
            item_weights = [int(n) for n in data[3]]   
            x = [int(n) for n in data[4]]   
            y = [int(n) for n in data[5]]  
            instances[inst.name]['M'] = M 
            instances[inst.name]['N'] = N 
            instances[inst.name]['max_loads'] = max_loads 
            instances[inst.name]['item_weights'] = item_weights
            instances[inst.name]['dx']= [x[-1]] + x[0 : len(x)-1]
            instances[inst.name]['dy']= [y[-1]] + y[0 : len(y)-1]
            #distance-matrix, manhattan distance, order n+1
            instances[inst.name]['distance_matrix'] = np.array(
                [[abs(x[i]-x[j])+abs(y[i]-y[j]) for i in range(N+1)] for j in range(N+1)]) 

In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial.distance import cityblock
from gurobipy import Model, GRB, quicksum


inst_to_run = "inst01"

n = instances[inst_to_run]["N"]                                 # number of clients  
m = instances[inst_to_run]["M"]                                 # umber of couriers                                             
xc = instances[inst_to_run]["dx"]                               # x-coordinate of clients and depot set as first element
yc = instances[inst_to_run]["dy"]                               # y-coordinate of clients and depot set as first element

N = [i for i in range(1,n+1)]                                   # Customers                            
V = [0] + N                     
A = [(i, j) for i in V for j in V if i != j]                    # Edges between each pair of points 
weight = [0]+instances[inst_to_run]["item_weights"]             # Weight for each item   
Capacities = instances[inst_to_run]["max_loads"]                # Couriers capacities 
Dist = {}                                                       # Distances between points
for i in range(n+1):
    for j in range(n+1):
        Dist[(i,j)] = cityblock((xc[i],yc[i]),(xc[j],yc[j]))                            
Couriers_encoding = [i for i in range(m)]                          
courier_o_h_e = {}                                              # Dictionary of one hot encoding of the information about couriers 
aux_vars = {}                                                                                             
 
mdl = Model('CVRP')
x = mdl.addVars(A, vtype=GRB.BINARY)                                        # Decision variable for whether an edge is chosen.
l = mdl.addVars(V, lb=0, ub=n, vtype=GRB.INTEGER)                           # Order/rank of the point               
for i in range(1,n+1):
    courier_o_h_e[i] = mdl.addVars(Couriers_encoding, vtype=GRB.BINARY)     # One hot encoders for courier associated with a given point 
    aux_vars[i] = mdl.addVars(Couriers_encoding, vtype=GRB.INTEGER) 
    
mdl.modelSense = GRB.MINIMIZE
mdl.setObjective(quicksum(x[i, j] * Dist[i, j] for (i, j) in A))            # Objective function

mdl.addConstrs(
    quicksum(x[i,j] for j in V if j != i) == 1 for i in N)                  # Only one connection into point i
mdl.addConstrs(
    quicksum(x[i,j] for i in V if i != j) == 1 for j in N)                  # Only one edge out of point i
mdl.addConstr(
    quicksum(x[0,j] for j in N ) <= m )                                     # No more than "m" departures from the depot

mdl.addConstr(
    quicksum(x[0,j] for j in N ) == quicksum(x[j,0] for j in N )            # Implied constraint: Same number of couriers leaving and arriving to the depot
)

mdl.addConstr(l[0] == 1)                                                    # Sub-tour elimination
mdl.addConstrs(
    (l[i] + x[i,j]) <= (l[j] + n * (1 - x[i,j])) for i,j in A if j!=0 )

for k in range(m):
    mdl.addConstrs((aux_vars[i][k] <= courier_o_h_e[i][k]) for i in N)                                  # Double-tour elimination
    mdl.addConstrs((aux_vars[i][k] <= x[0,i]) for i in N )
    mdl.addConstrs((aux_vars[i][k] >= courier_o_h_e[i][k] + x[0,i] - 1 ) for i in N )
    mdl.addConstr(
        quicksum(aux_vars[i][k] for i in N) <= 1 
    )

for k in range(m):                                                                                      # If x[i,j] = True, i,j share the same courier
    for i,j in A:
        if i!=0 and j!=0:
            mdl.addConstr(x[i,j] + courier_o_h_e[i][k] - courier_o_h_e[j][k] <= 1 )
            mdl.addConstr(x[i,j] - courier_o_h_e[i][k] + courier_o_h_e[j][k] <= 1 )

mdl.addConstrs(
    quicksum(courier_o_h_e[i][k] for k in range(m)) == 1 for i in N)                                                    
    
mdl.addConstrs(
    quicksum( weight[i] * courier_o_h_e[i][k] for i in N ) <= Capacities[k] for k in range(m))          # Courier k carries less weight than its own capacity



mdl.params.ScaleFlag = 1
mdl.params.MIPFocus = 1 
mdl.Params.TimeLimit = 300
mdl.optimize()

In [ ]:
tour = {}

for i in N :
    try :
        x[(0,i)].x> 0.1
    except AttributeError:
        print("Total distance: N/A")
        break
    if x[(0,i)].x > 0.9 :
        for k in range(m):
            if int(str(courier_o_h_e[i][k])[-6:-4]) >0 :
                temp = k
        temp = temp +1
        tour[temp] = []
        aux = [0,i]
        while i!=0:
            j=i
            for k  in V :
                if j!=k and x[(j,k)].x >0.9:
                    aux.append(k)
                    i = k
        tour[temp].append(aux)

for key in tour.keys():
    temp = str(tour[key])
    temp = temp.replace("[","")
    temp = temp.replace("]","")
    temp = temp.replace(", "," => ")
    tour[key] = temp

keys = [i for i in tour.keys()]
print("Tour planned:")

for key in sorted(keys):
    print("Courier "+str(key))
    print(tour[key])

print("---------------------------")
try:
    print("Total distance:",int(mdl.ObjVal))
except OverflowError:
    print("N/A")